# Simple Version 00

This notebook is an attempt to create a very simple version of the finite algebra code.

In [6]:
import numpy as np
import itertools as it

## Cayley Table

*"Named after the 19th-century British mathematician Arthur Cayley, a Cayley table describes the structure of a finite group by arranging all the possible products of all the group's elements in a square table reminiscent of an addition or multiplication table. Many properties of a group – such as whether or not it is abelian, which elements are inverses of which elements, and the size and contents of the group's center – can be discovered from its Cayley table."* [Wikipedia](https://en.wikipedia.org/wiki/Cayley_table)

Here, the **CayleyTable** class is used to represent a square array of indices, where the indices reference the positions of each of an algebra's elements within the list of all elements. For example, if there are 3 elements, [e, a, aa], then the corresponding instance of a CayleyTable will contain a 3x3 array where each array element is either a 0, 1, or 2.

In [7]:
class CayleyTable:
    
    def __init__(self, array):
        tmp = np.array(array, dtype=int)
        nrows, ncols = tmp.shape
        if nrows == ncols:
            if (np.min(tmp) >= 0) and (np.max(tmp) < nrows):
                self.__table = tmp
            else:
                raise ValueError(f"Array elements must be integers between 0 and {nrows - 1}, inclusive.")
        else:
            raise ValueError(f"The array must be square.")
    
    def __repr__(self):
        return f"{self.__class__.__name__}({self.__table.tolist()})"
    
    def __getitem__(self, tup):
        row, col = tup
        return self.__table[row][col]
    
    @property
    def size(self):
        return self.__table.shape[0]
    
    @property
    def array(self):
        return self.__table

In [8]:
arr0 = [[0, 1, 2], [1, 2, 0], [2, 0, 1]]
arr0

[[0, 1, 2], [1, 2, 0], [2, 0, 1]]

In [9]:
arr1 = [[0, 1, 2], [1, 2, 0]]
try:
    CayleyTable(arr1)
except Exception as msg:
    print(msg)

The array must be square.


In [10]:
arr2 = [[0, 1, 2], [1, 7, 0], [2, 0, 1]]
try:
    CayleyTable(arr2)
except Exception as msg:
    print(msg)

Array elements must be integers between 0 and 2, inclusive.


In [11]:
cg3 = CayleyTable(arr0)
cg3

CayleyTable([[0, 1, 2], [1, 2, 0], [2, 0, 1]])

In [12]:
cg3.array

array([[0, 1, 2],
       [1, 2, 0],
       [2, 0, 1]])

In [14]:
cg3.size

3

## Binary Operator

The **BinaryOperator** class brings the list of elements (as strings) together with the corresponding CayleyTable.

In [15]:
class BinaryOperator:
    
    def __init__(self, elements, cayley_table):
        self.__elements = elements
        self.__table = cayley_table
    
    def __call__(self, elem1, elem2):
        row = self.__elements.index(elem1)
        col = self.__elements.index(elem2)
        index = self.__table[row, col]
        return self.__elements[index]
    
    @property
    def elements(self):
        return self.__elements
    
    @property
    def table(self):
        return self.__table

In [16]:
elems = ['e', 'a', 'aa']

In [17]:
binop = BinaryOperator(elems, cg3)

In [18]:
binop('e', 'a')

'a'

In [19]:
binop('a', 'a')

'aa'

In [20]:
binop('a', 'aa')

'e'

In [21]:
binop.elements

['e', 'a', 'aa']

In [22]:
binop.table

CayleyTable([[0, 1, 2], [1, 2, 0], [2, 0, 1]])

In [23]:
binop.table.array

array([[0, 1, 2],
       [1, 2, 0],
       [2, 0, 1]])

## Algebra

An **Algebra** consists of a set of elements with an associated binary operator, usually referred to as "multiplication" or "addition", depending on whether the operator is represented as $\times$ or $+$, resp.

In [24]:
class Algebra:
    
    def __init__(self, elements, array):
        self.__binop = BinaryOperator(elements, CayleyTable(array))
        
    def __getitem__(self, index):
        return self.__binop.elements[index]
    
    @property
    def elements(self):
        return self.__binop.elements
    
    @property
    def order(self):
        return len(self.elements)
    
    @property
    def table(self):
        return self.__binop.table
    
    def __repr__(self):
        return f"Algebra(\n{self.elements},\n {self.table.array.tolist()})"
    
    def op(self, elem1, elem2):
        """Use the algebra's binary operation to add/multiply two of it's elements,
        and return the sum/product.
        """
        return self.__binop(elem1, elem2)
    
    def __mul__(self, other):
        """Return the direct product of self with other. Other must be an algebra
        """
        dp_elements = list(it.product(self.elements, other.elements))  # cross-product of elements
        dp_table = list()  # start a new table
        for a in dp_elements:
            dp_table_row = list()  # Start a new row
            for b in dp_elements:
                dp_table_row.append(dp_elements.index((self.op(a[0], b[0]), other.op(a[1], b[1]))))
            dp_table.append(dp_table_row)  # Append the new row to the table
        return Algebra(list([f"{elem[0]}:{elem[1]}" for elem in dp_elements]),
                       dp_table)

In [25]:
alg0 = Algebra(elems, arr0)

In [26]:
alg0.elements

['e', 'a', 'aa']

In [27]:
alg0.order

3

In [28]:
alg0.table

CayleyTable([[0, 1, 2], [1, 2, 0], [2, 0, 1]])

In [29]:
alg0

Algebra(
['e', 'a', 'aa'],
 [[0, 1, 2], [1, 2, 0], [2, 0, 1]])

In [30]:
alg0.op('a', 'aa')

'e'

In [31]:
alg1 = alg0 * alg0
alg1

Algebra(
['e:e', 'e:a', 'e:aa', 'a:e', 'a:a', 'a:aa', 'aa:e', 'aa:a', 'aa:aa'],
 [[0, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 0, 4, 5, 3, 7, 8, 6], [2, 0, 1, 5, 3, 4, 8, 6, 7], [3, 4, 5, 6, 7, 8, 0, 1, 2], [4, 5, 3, 7, 8, 6, 1, 2, 0], [5, 3, 4, 8, 6, 7, 2, 0, 1], [6, 7, 8, 0, 1, 2, 3, 4, 5], [7, 8, 6, 1, 2, 0, 4, 5, 3], [8, 6, 7, 2, 0, 1, 5, 3, 4]])

In [32]:
alg1.order

9

## Elements

In [33]:
class Element:
    
    def __init__(self, elem_name, algebra):
        self.__algebra = algebra
        if isinstance(elem_name, str):
            if elem_name in self.__algebra:
                self.__name = elem_name
            else:
                raise ValueError(f"name must be an element of algebra")
        else:
            raise ValueError(f"name must be a string")

    def __repr__(self):
        return repr(self.__name)
    
    @property
    def name(self):
        return self.__name
    
    def __add__(self, other):
        elem = self.__algebra.op(self.__name, other.name)
        return Element(elem, self.__algebra)

In [34]:
def element_map(algebra):
    """Returns a dictionary where element names (str) are keys and the corresponding
    Element instances are the values."""
    return {elem: Element(elem, algebra) for elem in algebra.elements}

In [35]:
class Context:

    def __init__(self, algebra):
        self.element_map = element_map(algebra)

    def __enter__(self):
        return self.element_map

    def __exit__(self, _type, value, traceback):
        pass

In [36]:
with Context(alg1) as A:
    print(A['a:aa'] + A['aa:aa'] + A['a:a'])

'a:aa'
